In [1]:
import os
print(os.getcwd())
def update_working_directory():
    from pathlib import Path
    p = Path(os.getcwd()).parents[0]
    os.chdir(p)
    print(p)
update_working_directory()

/Users/admin/Projects/doggo/notebooks
/Users/admin/Projects/doggo


# Imports

In [2]:
# -*- coding: utf-8 -*-
import dash
import dill
import pandas as pd
import json
import numpy as np
import pandas as pd

import dash_core_components as dcc
import dash_html_components as html
from src.utils.utils_dict import flatten

from src.utils.maths import get_avg_n_points

In [3]:
method_id = 'sarsa_test'

In [4]:
params_method = json.loads(open('src/models/value_based/sarsa/config/{}.json'.format(method_id)).read())
params_method

{'method': 'SARSA',
 'epsilon': {'init_epsilon': 1,
  'decay_epsilon': 'per_state',
  'min_epsilon': 0},
 'alpha': {'init_alpha': 0.5, 'decay_alpha': 'fixed', 'min_alpha': 0.5},
 'init_Q_type': 'ones',
 'n_episodes': 1000,
 'nmax_steps': 1440,
 'gamma': 0.95,
 'start_at_random': False,
 'KLdiv_convergence': 1e-10}

# Functions

In [5]:
def flatten(d, parent_key='', sep='__'):
    items = []
    for k, v in d.items():
        new_key = parent_key + sep + k if parent_key else k
        if isinstance(v, collections.abc.MutableMapping):
            items.extend(flatten(v, new_key, sep=sep).items())
        else:
            items.append((new_key, v))
    return dict(items)


def running_mean(x, n_points):
    cumsum = np.cumsum(np.insert(x, 0, 0))
    return (cumsum[n_points:] - cumsum[:-n_points]) / float(n_points)


def get_avg_n_points(list_points, n_points):
    n_moving_points = int(np.ceil(len(list_points) / n_points))
    y_all = running_mean(list_points, n_moving_points)
    y = [list_points[0]] + [j for i, j in enumerate(y_all) if i % n_moving_points == 0] + [y_all[-1]]
    return y


def get_available_methods():
    my_path = 'models'
    available_methods_unsorted = [f.split('__')[0] for f in listdir(my_path) if isfile(join(my_path, f)) & ('evo_training' in f)]
    return np.sort(available_methods_unsorted)


def generate_table(dataframe, max_rows=10):
    return html.Table([
        html.Thead(
            html.Tr([html.Th(col) for col in dataframe.columns])
        ),
        html.Tbody([
            html.Tr([
                html.Td(dataframe.iloc[i][col]) for col in dataframe.columns
            ]) for i in range(min(len(dataframe), max_rows))
        ])
    ])

# Table parameters

In [ ]:
method_id = 'sarsa_test'

In [ ]:
params_method = json.loads(open('src/models/value_based/sarsa/config/{}.json'.format(method_id)).read())
params_method['method_id'] = method_id
flatten(params_method)

In [ ]:
parameters_table = pd.DataFrame()
parameters_table = parameters_table.append(flatten(params_method), ignore_index=True)
parameters_table

In [ ]:
parameters_table = parameters_table[[
    'method', 'method_id',
    'n_episodes', 'nmax_steps', 'gamma', 'start_at_random',
    'epsilon__init_epsilon', 'epsilon__decay_epsilon', 'epsilon__min_epsilon',
    'alpha__init_alpha', 'alpha__decay_alpha', 'alpha__min_alpha',
    'init_Q_type', 'KLdiv_convergence'
]]

In [ ]:
parameters_table

# Reinforcement Learning

In [6]:
def fetch_data_evo_real_episode(method_id):

    with open("models/{}__evolution_real_episode.pkl".format(method_id), "rb") as input_file:
        evolution_real_episode = dill.load(input_file)

    evolution_real_episode_reduced = dict()
    for i_key in evolution_real_episode.keys():
        if i_key == 'cause_of_death':
            evolution_real_episode_reduced[i_key] = evolution_real_episode[i_key]
        else:
            evolution_real_episode_reduced[i_key] = get_avg_n_points(evolution_real_episode[i_key], n_points=100)

    return evolution_real_episode_reduced

evolution_real_episode_reduced = fetch_data_evo_real_episode(method_id)

In [7]:
evolution_real_episode_reduced.keys()

dict_keys(['avg_reward', 'sum_reward', 'n_steps', 'avg_happiness', 'sum_happiness', 'n_actions', 'cause_of_death', 'avg_food', 'sum_food', 'avg_inv_fat', 'sum_inv_fat', 'avg_affection', 'sum_affection'])

# During training

In [ ]:
def fetch_data_evo_training(method_id):

    with open("models/{}__evo_training.pkl".format(method_id), "rb") as input_file:
        evo_training = dill.load(input_file)
    evo_training = flatten(evo_training)
    
    evo_training_reduced = dict()
    for i_key in evo_training.keys():
        if i_key in ['evo_avg_reward_per_step', 'evo_min_reward_per_step', 'evo_max_reward_per_step', 'evo_n_steps', 'evo_avg_happiness', 'evo_min_happiness', 'evo_max_happiness', 'checking__evo_epsilon', 'checking__evo_alpha']:
            evo_training_reduced[i_key] = get_avg_n_points(evo_training[i_key], n_points=10)

    return evo_training_reduced

In [ ]:
fetch_data_evo_training(method_id).keys()

# Cause of death

In [8]:
n_points = 10

In [9]:
evolution_real_episode_reduced['cause_of_death']

['fat',
 'food',
 'food',
 'food',
 'food',
 'fat',
 'food',
 'fat',
 'food',
 'food',
 'food',
 'food',
 'food',
 'fat',
 'food',
 'food',
 'food',
 'food',
 'food',
 'food',
 'food',
 'food',
 'food',
 'fat',
 'food',
 'food',
 'food',
 'food',
 'food',
 'food',
 'food',
 'fat',
 'fat',
 'fat',
 'fat',
 'fat',
 'fat',
 'food',
 'food',
 'food',
 'food',
 'food',
 'food',
 'food',
 'food',
 'fat',
 'food',
 'food',
 'food',
 'fat',
 'food',
 'food',
 'fat',
 'fat',
 'food',
 'food',
 'food',
 'food',
 'food',
 'food',
 'no_death',
 'food',
 'food',
 'food',
 'fat',
 'food',
 'food',
 'food',
 'food',
 'food',
 'food',
 'food',
 'food',
 'food',
 'food',
 'food',
 'food',
 'food',
 'fat',
 'food',
 'food',
 'food',
 'food',
 'food',
 'food',
 'food',
 'food',
 'food',
 'food',
 'food',
 'food',
 'food',
 'food',
 'food',
 'fat',
 'food',
 'fat',
 'food',
 'food',
 'food',
 'food',
 'food',
 'food',
 'fat',
 'food',
 'food',
 'food',
 'fat',
 'food',
 'food',
 'food',
 'food',
 'food',


In [14]:
from more_itertools import divide
cause_of_death_divided = [list(l) for l in divide(n_points, evolution_real_episode_reduced['cause_of_death'])]

In [22]:
cause_of_death_pct = {
    'food': [],
    'fat': [],
    'affection': [],
    'no_death': []
}

for i_list in cause_of_death_divided:
    cause_of_death_pct['food'].append(int(np.floor(len([x for x in i_list if x == 'food'])/len(i_list)*100)))
    cause_of_death_pct['fat'].append(int(np.floor(len([x for x in i_list if x == 'fat'])/len(i_list)*100)))
    cause_of_death_pct['affection'].append(int(np.floor(len([x for x in i_list if x == 'affection'])/len(i_list)*100)))
    cause_of_death_pct['no_death'].append(int(np.floor(len([x for x in i_list if x == 'no_death'])/len(i_list)*100)))

In [21]:
int(np.floor(len([x for x in i_list if x == 'fat'])/len(i_list)*100))

19

In [23]:
cause_of_death_pct

{'food': [80, 74, 86, 69, 45, 6, 11, 17, 24, 7],
 'fat': [19, 24, 14, 30, 53, 85, 87, 78, 76, 84],
 'affection': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 'no_death': [1, 2, 0, 1, 2, 9, 2, 5, 0, 9]}

In [35]:
import plotly.graph_objects as go
x = list(range(n_points))

fig = go.Figure()
fig.add_trace(go.Bar(x=x, y=cause_of_death_pct['no_death'], name='no_death', width=0.95, marker_color=['tan',]*n_points))
fig.add_trace(go.Bar(x=x, y=cause_of_death_pct['affection'], name='affection', width=0.95))
fig.add_trace(go.Bar(x=x, y=cause_of_death_pct['fat'], name='fat', width=0.95))
fig.add_trace(go.Bar(x=x, y=cause_of_death_pct['food'], name='food', width=0.95))


#fig.update_layout(barmode='relative', title_text='Relative Barmode')
fig.update_layout(barmode='stack', xaxis={'categoryorder':'category ascending'})
fig.show()

# One episode

In [37]:
def fetch_data_evo_episode(method_id):

    with open("models/{}__evo_episode.pkl".format(method_id), "rb") as input_file:
        evo_episode = dill.load(input_file)
    
    return evo_episode

In [41]:
evo_episode = fetch_data_evo_episode(method_id)
evo_episode.keys()

dict_keys(['n_episode_steps', 'done', 'action', 'action_taken', 'reward', 'happiness', 'food', 'inv_fat', 'affection', 'avg_reward', 'n_steps', 'avg_happiness'])

In [70]:
import plotly.graph_objects as go
import numpy as np

x = np.arange(len(evo_episode['happiness']))

# Create traces
fig = go.Figure()

fig.add_trace(go.Scatter(
    x=x, y=evo_episode['happiness'],
    legendgroup="happiness",
    mode='lines',
    name='happiness',
    marker_color='black', line={'width':2}
))

fig.add_trace(go.Scatter(
    x=x, y=evo_episode['food'],
    legendgroup="characteristics",
    mode='lines',
    name='food',
    marker_color='lightgreen', line={'width':1}
))

fig.add_trace(go.Scatter(
    x=x, y=evo_episode['inv_fat'],
    legendgroup="characteristics",
    mode='lines',
    name='inv. fat',
    marker_color='lightcoral', line={'width':1}
))

fig.add_trace(go.Scatter(
    x=x, y=evo_episode['affection'],
    legendgroup="characteristics",
    mode='lines',
    name='affection',
    marker_color='lightblue', line={'width':1}
))

action_dict = {
    0: ['NO ACTION', 'white'],
    1: ['WALKING', 'indigo'],
    2: ['EATING', 'hotpink'],
    3: ['PLAYING', 'darkorange']
}

fig.add_trace(go.Scatter(
    x=[0,0], y=[0,1],
    legendgroup="walking",
    mode='lines',
    name=action_dict[1][0] + ' - taken',
    marker_color=action_dict[1][1],
    line={'width':1, 'dash':'solid'}
))

fig.add_trace(go.Scatter(
    x=[0,0], y=[0,1],
    legendgroup="walking",
    mode='lines',
    name=action_dict[1][0] + ' - non-taken',
    marker_color=action_dict[1][1],
    line={'width':1, 'dash':'dot'}
))

fig.add_trace(go.Scatter(
    x=[0,0], y=[0,1],
    legendgroup="eating",
    mode='lines',
    name=action_dict[2][0] + ' - taken',
    marker_color=action_dict[2][1],
    line={'width':1, 'dash':'solid'}
))

fig.add_trace(go.Scatter(
    x=[0,0], y=[0,1],
    legendgroup="eating",
    mode='lines',
    name=action_dict[2][0] + ' - non-taken',
    marker_color=action_dict[2][1],
    line={'width':1, 'dash':'solid'}
))

fig.add_trace(go.Scatter(
    x=[0,0], y=[0,1],
    legendgroup="playing",
    mode='lines',
    name=action_dict[3][0] + ' - taken',
    marker_color=action_dict[3][1],
    line={'width':1, 'dash':'solid'}
))

fig.add_trace(go.Scatter(
    x=[0,0], y=[0,1],
    legendgroup="playing",
    mode='lines',
    name=action_dict[3][0] + ' - non-taken',
    marker_color=action_dict[3][1],
    line={'width':1, 'dash':'solid'}
))

        
for i in range(len(evo_episode['action'])):
    
    if evo_episode['action'][i] != 0:
        
        fig.add_trace(go.Scatter(
            x=[i,i], y=[0,1],
            mode='lines',
            name=action_dict[evo_episode['action'][i]][0],
            marker_color=action_dict[evo_episode['action'][i]][1],
            line={'width':0.5, 'dash':'dot'},
            showlegend=False
        ))
        
    if evo_episode['action_taken'][i] != 0:
        
        fig.add_trace(go.Scatter(
            x=[i,i], y=[0,1],
            mode='lines',
            name=action_dict[evo_episode['action_taken'][i]][0],
            marker_color=action_dict[evo_episode['action_taken'][i]][1],
            line={'width':0.5, 'dash':'solid'},
            showlegend=False
        ))
        

fig.update_layout(
    title={
        'text': "One episode",
        'xanchor': 'center',
        'yanchor': 'top'},
    xaxis_title="Steps",
    yaxis_title="State",
    legend= {'itemsizing': 'constant'}
)



fig.show()

# Progress bar

In [72]:
df

,total_bill,tip,sex,smoker,day,time,size
0,16.99,1.01,Female,No,Sun,Dinner,2
1,10.34,1.66,Male,No,Sun,Dinner,3
2,21.01,3.50,Male,No,Sun,Dinner,3
3,23.68,3.31,Male,No,Sun,Dinner,2
4,24.59,3.61,Female,No,Sun,Dinner,4
...,...,...,...,...,...,...,...
239,29.03,5.92,Male,No,Sat,Dinner,3
240,27.18,2.00,Female,Yes,Sat,Dinner,2
241,22.67,2.00,Male,Yes,Sat,Dinner,2
242,17.82,1.75,Male,No,Sat,Dinner,2


In [87]:
import plotly.graph_objects as go

progress_value = 5440
max_value = 10000

if progress_value/max_value < 0.33:
    color='lightcoral'
elif progress_value/max_value < 0.66:
    color='lightgoldenrodyellow'
else:
    color='lightgreen'

fig = go.Figure(go.Bar(
            x=[progress_value],
            y=['progress'],
            orientation='h',
            marker=dict(color=color)
))

fig.update_layout(
    xaxis=dict(range=[0, max_value])
)

fig.show()

In [101]:
list_evo_training = {
    'sarsa_test': {
        'evo_avg_reward_per_step': list(range(0,10)),
        'n_episodes': 40
    },
    'sarsa_test2': {
        'evo_avg_reward_per_step': list(range(0,20)),
        'n_episodes': 40
    }
}


In [106]:
max_value_overall = 0

fig = go.Figure()

for i_method_id in list_evo_training.keys():

    progress_value = len(list_evo_training[i_method_id]['evo_avg_reward_per_step'])
    max_value = list_evo_training[i_method_id]['n_episodes']

    if progress_value/max_value < 0.33:
        color_progress='lightcoral'
    elif progress_value/max_value < 0.66:
        color_progress='lightgoldenrodyellow'
    else:
        color_progress='lightgreen'
    
    fig.add_trace(go.Bar(
        x=list(range(progress_value)),
        y=[i_method_id],
        name='progress',
        orientation='h',
        marker=dict(
            color=list(range(progress_value)),
            colorbar=dict(
                title="Colorbar"
            ),
            colorscale="Viridis"
        )
    ))
        
    max_value_overall = max(max_value_overall, max_value)

fig.update_layout(
    xaxis=dict(range=[0, max_value_overall])
)

fig.show()

In [107]:
import plotly.express as px

fig = px.colors.sequential.swatches()
fig.show()

In [112]:
plotly.colors.sequential.Redor

NameError: name 'plotly' is not defined